# 数据扩增
在赛题中我们需要对的图像进行字符识别，因此需要我们完成的数据的读取操作，同时也需要完成数据扩增（Data Augmentation）操作。

在深度学习中数据扩增方法非常重要
数据扩增可以增加训练集的样本，同时也可以有效缓解模型过拟合的情况，也可以给模型带来的更强的泛化能力。


从颜色空间、尺度空间到样本空间，同时根据不同任务数据扩增都有相应的区别。
对于图像分类，数据扩增一般不会改变标签；对于物体检测，数据扩增会改变物体坐标位置；对于图像分割，数据扩增会改变像素标签。

## 常见的数据扩增方法
在常见的数据扩增方法中，一般会从图像颜色、尺寸、形态、空间和像素等角度进行变换。当然不同的数据扩增方法可以自由进行组合，得到更加丰富的数据扩增方法。

以torchvision为例，常见的数据扩增方法包括：

transforms.CenterCrop 对图片中心进行裁剪
transforms.ColorJitter 对图像颜色的对比度、饱和度和零度进行变换
transforms.FiveCrop 对图像四个角和中心进行裁剪得到五分图像
transforms.Grayscale 对图像进行灰度变换
transforms.Pad 使用固定值进行像素填充
transforms.RandomAffine 随机仿射变换
transforms.RandomCrop 随机区域裁剪
transforms.RandomHorizontalFlip 随机水平翻转
transforms.RandomRotation 随机旋转
transforms.RandomVerticalFlip 随机垂直翻转
赛题任务是需要对图像中的字符进行识别，因此对于字符图片并不能进行翻转操作。

## 常用的数据扩增库
torchvision
https://github.com/pytorch/vision
pytorch官方提供的数据扩增库，提供了基本的数据数据扩增方法，可以无缝与torch进行集成；但数据扩增方法种类较少，且速度中等；

imgaug
https://github.com/aleju/imgaug
imgaug是常用的第三方数据扩增库，提供了多样的数据扩增方法，且组合起来非常方便，速度较快；

albumentations
https://albumentations.readthedocs.io
是常用的第三方数据扩增库，提供了多样的数据扩增方法，对图像分类、语义分割、物体检测和关键点检测都支持，速度较快

# 用Pytorch读取赛题数据
在Pytorch中数据是通过Dataset进行封装，并通过DataLoder进行并行读取。所以我们只需要重载一下数据读取的逻辑就可以完成数据的读取。

In [8]:
import os, sys, glob, shutil, json
import cv2

from PIL import Image
import numpy as np

import torch
from torch.utils.data.dataset import Dataset
import torchvision.transforms as transforms

class SVHNDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None

    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        # 原始SVHN中类别10为数字0
        lbl = np.array(self.img_label[index], dtype=np.int)
        lbl = list(lbl)  + (5 - len(lbl)) * [10]
        return img, torch.from_numpy(np.array(lbl[:5]))

    def __len__(self):
        return len(self.img_path)

train_path = glob.glob('./input/train/*.png')
train_path.sort()
# print(train_path)
train_json = json.load(open('./input/train.json'))
train_label = [train_json[x]['label'] for x in train_json]

如果图中数字为1，9的话，通过以上方法在遍历图片时可以得到[1,9,10,10,10]

In [9]:
data = SVHNDataset(train_path, train_label,
          transforms.Compose([
              # 缩放到固定尺寸
              transforms.Resize((64, 128)),

              # 随机颜色变换
              transforms.ColorJitter(0.2, 0.2, 0.2),

              # 加入随机旋转
              transforms.RandomRotation(5),

              # 将图片转换为pytorch 的tesntor
              transforms.ToTensor(),

              # 对图像像素进行归一化
              transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ]))
#使用DataLoader封装数据
train_loader = torch.utils.data.DataLoader(
       data,
    batch_size=10, # 每批样本个数
    shuffle=False, # 是否打乱顺序
    num_workers=10, # 读取的线程个数
)


In [10]:
for x,y in data:
    print(x,x.shape)
    print(y.shape)
    break


tensor([[[-0.6281, -0.5767, -0.6794,  ...,  1.3755,  1.3755,  1.3755],
         [-0.5596, -0.5424, -0.5938,  ...,  1.3755,  1.3755,  1.3755],
         [-0.4397, -0.4397, -0.5424,  ...,  1.3755,  1.3755,  1.3755],
         ...,
         [-2.0323, -2.0494, -2.0323,  ...,  1.3755,  1.3755,  1.3755],
         [-2.0152, -1.9809, -1.9467,  ...,  1.3755,  1.3755,  1.3755],
         [-1.9980, -1.9467, -1.8782,  ...,  1.3755,  1.3755,  1.3755]],

        [[-0.4076, -0.4951, -0.4251,  ...,  1.5357,  1.5357,  1.5357],
         [-0.3550, -0.4251, -0.3901,  ...,  1.5357,  1.5357,  1.5357],
         [-0.2675, -0.2850, -0.2850,  ...,  1.5357,  1.5357,  1.5357],
         ...,
         [-1.9132, -1.9307, -1.9132,  ...,  1.5357,  1.5357,  1.5357],
         [-1.8957, -1.9132, -1.9832,  ...,  1.5357,  1.5357,  1.5357],
         [-1.8606, -1.9307, -1.9657,  ...,  1.5357,  1.5357,  1.5357]],

        [[-0.3404, -0.2184, -0.3055,  ...,  1.7511,  1.7511,  1.7511],
         [-0.0615, -0.0790, -0.1138,  ...,  1